In [11]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, datasets
import random
from typing import Any, Tuple
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")



# Example transforms (you can adjust as needed)
leak_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])

])

no_leak_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])

])

# Custom ImageFolder to apply different transforms based on label
class CustomImageFolder(datasets.ImageFolder):
    def __getitem__(self, index: int) -> Tuple[Any, Any]:
        path, label = self.samples[index]
        sample = self.loader(path)
        if label == 0:  # Assuming 0 is no-leak
            sample = no_leak_transform(sample)
        else:  # Assuming 1 is leak
            sample = leak_transform(sample)
        return sample, label

# Custom Dataset for handling class imbalance
class BalancedDataset(Dataset):
    def __init__(self, dataset: Dataset):
        self.dataset = dataset
        # Get indices for each class
        self.no_leak_indices = [i for i, (_, label) in enumerate(dataset) if label == 0]
        self.leak_indices = [i for i, (_, label) in enumerate(dataset) if label == 1]

        # Upsample minority class (leak)
        self.upsampled_leak_indices = random.choices(
            self.leak_indices,
            k=len(self.no_leak_indices)  # Match majority class size
        )

        # Combine indices
        self.indices = self.no_leak_indices + self.upsampled_leak_indices
        random.shuffle(self.indices)

    def __len__(self) -> int:
        return len(self.indices)

    def __getitem__(self, idx: int) -> Tuple[Any, Any]:
        return self.dataset[self.indices[idx]]

# Load the dataset

class CustomImageFolder(datasets.ImageFolder):
    def __getitem__(self, index):
        path, label = self.samples[index]
        sample = self.loader(path)
        # Transformations happen HERE based on label
        if label == 0:
            sample = no_leak_transform(sample)
        else:
            sample = leak_transform(sample)
        return sample, label

dataset = CustomImageFolder(root=r'D:\3IA\Leak_detection\data')

# Split dataset into train, validation, and test sets
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(
    dataset, [train_size, val_size, test_size]
)

# Create balanced training dataset
balanced_train_dataset = BalancedDataset(train_dataset)
train_loader = DataLoader(balanced_train_dataset, batch_size=32, shuffle=True)

# For validation, you might want to keep original distribution
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Test DataLoader (keep original distribution)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Calculate class weights (inverse frequency)
num_leak = len([label for _, label in dataset if label == 1])
num_no_leak = len([label for _, label in dataset if label == 0])
total_samples = num_leak + num_no_leak
weight_no_leak = total_samples / (2 * num_no_leak)
weight_leak = total_samples / (2 * num_leak)
class_weights = torch.tensor([weight_no_leak, weight_leak])

print(f"Training samples: {len(balanced_train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")
print(f"Test samples: {len(test_dataset)}")
print(f"Class weights: {class_weights}")
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
import numpy as np

class ImprovedLeakDetectionCNN(nn.Module):
    def __init__(self):
        super(ImprovedLeakDetectionCNN, self).__init__()
        # Convolutional blocks with progressive dropout
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.dropout1 = nn.Dropout2d(0.2)  # Increased from 0.1
        
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.dropout2 = nn.Dropout2d(0.3)  # Increased from 0.2
        
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.dropout3 = nn.Dropout2d(0.4)  # Increased from 0.3
        
        # Reduced channels in final conv layer
        self.conv4 = nn.Conv2d(128, 192, kernel_size=3, padding=1)  # Reduced from 256
        self.bn4 = nn.BatchNorm2d(192)
        self.dropout4 = nn.Dropout2d(0.5)  # Increased from 0.4

        self.pool = nn.MaxPool2d(2, 2)
        
        # Fully connected layers with higher dropout
        self.fc1 = nn.Linear(192 * 8 * 8, 384)  # Reduced from 512
        self.fc1_dropout = nn.Dropout(0.6)  # Increased from 0.5
        self.fc2 = nn.Linear(384, 2)

    def forward(self, x):
        # Conv block 1
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.dropout1(x)
        
        # Conv block 2
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.dropout2(x)
        
        # Conv block 3
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.dropout3(x)
        
        # Conv block 4
        x = self.pool(F.relu(self.bn4(self.conv4(x))))
        x = self.dropout4(x)

        # Fully connected
        x = x.view(-1, 192 * 8 * 8)
        x = self.fc1_dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x
# Initialize with weight decay (L2 regularization)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ImprovedLeakDetectionCNN().to(device)
# 1. Enhanced Focal Loss (handles class imbalance better)
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0):
        super(FocalLoss, self).__init__()
        self.alpha = torch.tensor([alpha, 1 - alpha])  # Ensure it matches class count
        self.gamma = gamma  

    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce_loss)

        # Convert targets to one-hot
        targets_onehot = F.one_hot(targets, num_classes=inputs.shape[1]).float()

        # Ensure alpha is correctly shaped
        alpha = self.alpha.to(inputs.device)[targets]  # Get per-example alpha values

        loss = (alpha * (1 - pt) ** self.gamma * ce_loss).mean()
        return loss

# Adjust weights for your dataset (No-Leak: 0.6, Leak: 0.4)
criterion = FocalLoss(alpha=0.25)


# 2. Optimizer with stronger L2 regularization
optimizer = optim.AdamW(
    model.parameters(),
    lr=0.001,
    weight_decay=1e-3  # Increased regularization strength
)

# 3. Cosine Annealing with Warm Restarts (better than basic CosineAnnealing)
scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
    optimizer,
    T_0=10,           # Number of epochs for first restart
    T_mult=2,         # Factor to increase T_0 after each restart
    eta_min=1e-5      # Minimum learning rate
)

# 4. Early Stopping Implementation (add this class)
class EarlyStopping:
    def __init__(self, patience=5, delta=0, verbose=True):
        self.patience = patience
        self.delta = delta
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf

    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.counter = 0
# The training loop remains the same as before
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=25):
    best_val_loss = float('inf')
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        train_progress = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Train]')
        for inputs, labels in train_progress:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            train_progress.set_postfix(
                loss=loss.item(),
                accuracy=100.*correct/total,
                lr=optimizer.param_groups[0]['lr']
            )

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = 100. * correct / total

        # Validation phase
        val_loss, val_acc = evaluate_model(model, val_loader, criterion)
        scheduler.step()

        print(f'Epoch {epoch+1}/{num_epochs} - '
              f'Train Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.2f}% - '
              f'Val Loss: {val_loss:.4f}, Acc: {val_acc:.2f}%')

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_loss': val_loss,
                'epoch': epoch
            }, 'best_model.pth')

    print('Training complete')
    checkpoint = torch.load('best_model.pth')
    model.load_state_dict(checkpoint['model_state_dict'])
    return model

# Evaluation function
def evaluate_model(model, data_loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    loss = running_loss / len(data_loader.dataset)
    accuracy = 100. * correct / total
    return loss, accuracy

# Test function
def test_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = 100. * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')

    # You can add more metrics here (precision, recall, F1, confusion matrix)
    from sklearn.metrics import classification_report, confusion_matrix
    print("\nClassification Report:")
    print(classification_report(all_labels, all_preds, target_names=['No Leak', 'Leak']))
    print("\nConfusion Matrix:")
    print(confusion_matrix(all_labels, all_preds))

    return accuracy

# Train the model
trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=25)

# Test the model
test_accuracy = test_model(trained_model, test_loader)

# Optionally: Load the best model for testing
# best_model = LeakDetectionCNN().to(device)
# best_model.load_state_dict(torch.load('best_model.pth'))
# test_accuracy = test_model(best_model, test_loader)

Using device: cuda
Training samples: 650
Validation samples: 153
Test samples: 154
Class weights: tensor([1.8494, 0.6853])


Epoch 1/25 [Train]: 100%|██████████| 21/21 [08:15<00:00, 23.61s/it, accuracy=78.2, loss=0.0959, lr=0.001] 


Epoch 1/25 - Train Loss: 0.7326, Acc: 78.15% - Val Loss: 0.0047, Acc: 98.69%


Epoch 2/25 [Train]: 100%|██████████| 21/21 [00:23<00:00,  1.11s/it, accuracy=96.6, loss=0.000437, lr=0.000976]


Epoch 2/25 - Train Loss: 0.0228, Acc: 96.62% - Val Loss: 0.0055, Acc: 98.69%


...

Epoch 24/25 - Train Loss: 0.0035, Acc: 97.85% - Val Loss: 0.0051, Acc: 98.04%


Epoch 25/25 [Train]: 100%|██████████| 21/21 [00:16<00:00,  1.30it/s, accuracy=97.5, loss=0.000416, lr=0.000214]


Epoch 25/25 - Train Loss: 0.0056, Acc: 97.54% - Val Loss: 0.0056, Acc: 97.39%
Training complete


C:\Users\hamza\AppData\Local\Temp\ipykernel_25668\2380555119.py:287: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('best_model.pth')


Test Accuracy: 98.70%

Classification Report:
              precision    recall  f1-score   support

     No Leak       1.00      0.95      0.98        44
        Leak       0.98      1.00      0.99       110

    accuracy                           0.99       154
   macro avg       0.99      0.98      0.98       154
weighted avg       0.99      0.99      0.99       154


Confusion Matrix:
[[ 42   2]
 [  0 110]]


In [13]:
torch.save(model.state_dict(), "water_leakage_detector_best.pth")
print("Model trained and saved!")

Model trained and saved!


In [15]:
train_loss, train_acc = evaluate_model(model, train_loader, criterion)
val_loss, val_acc = evaluate_model(model, val_loader, criterion)
test_loss, test_acc = evaluate_model(model, test_loader, criterion)

print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")
print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%")


Train Loss: 0.0057, Train Acc: 97.38%
Val Loss: 0.0041, Val Acc: 98.69%
Test Loss: 0.0039, Test Acc: 98.70%
